In [ ]:
# ライブラリのインストール
! pip install mosestokenizer
! pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
etest = list()
with open("/content/drive/MyDrive/Vdata/English/test.text.txt", "r") as f:
    for data in f:
        etest.append(data.rstrip('\n'))

In [ ]:
# 単語ベクトルの読み込み1
import gensim
model_path = "/content/drive/MyDrive/Vdata/crawl-300d-2M.vec"
w2v_1 = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=False)

In [ ]:
# 単語ベクトルの読み込み2
import gensim.downloader as api
w2v_2 = api.load("word2vec-google-news-300")

In [ ]:
# 前処理: 文 -> 単語リスト
from mosestokenizer import MosesTokenizer
tokenizer = MosesTokenizer("en", "no-escape")

s1 = list()
s2 = list()
tokenize1 = list()
tokenize2 = list()
stoplist1 = list()
stoplist2 = list()
wordslist1 = list()
wordslist2 = list()

# タブで分けてそれぞれs1とs2に入れる
textnum = 0
for sentence in etest:
    textnum += 1
    s = sentence.split("\t")
    s1.append(s[0])
    s2.append(s[1])

# 単語分割
def tokenize(s, tokenize):
    for sentence in s:
        tokenize.append(tokenizer(sentence.lower()))

# 単語を抜く
def stopwords(inlist, outlist):
    stop_words = ["a", "is", ".", ",", "s", "and", "by", "the", "are", "with", "on", "of", "at", "in", "up", "one", "two", "three", "for", "as", "out", "from", "very", "off", "to", "it", "t", ":", "ll", "will", "d", "do", "ve", "m", "she", "he", "her", "you", "we", "us", "your", "my", "me", "yours", "his", "him", "our", "'", ";", "'s", "i", '"']
    for sentence in inlist:
        outlist_stop = list()
        for word in sentence:
            if word not in stop_words:
                outlist_stop.append(word)
        outlist.append(outlist_stop)

# 数字置き換え
def replacenum(inlist, outlist):
    for sentence in inlist:
        replace = list()
        for word in sentence:
            if word.isdigit():
                replace.append("0")
            else:
                replace.append(word)
        outlist.append(replace)    

tokenize(s1, tokenize1)
tokenize(s2, tokenize2)

stopwords(tokenize1, wordslist1)
stopwords(tokenize2, wordslist2)

replacenum(stoplist1, wordslist1)
replacenum(stoplist2, wordslist2)

print(wordslist1)
print(wordslist2)

[['girl', 'styling', 'hair'], ['group', 'men', 'play', 'soccer', 'beach'], ['woman', 'measuring', 'another', 'woman', 'ankle'], ['man', 'cutting', 'cucumber'], ['man', 'playing', 'harp'], ['woman', 'cutting', 'onions'], ['man', 'riding', 'an', 'electric', 'bicycle'], ['man', 'playing', 'drums'], ['man', 'playing', 'guitar'], ['man', 'playing', 'guitar'], ['man', 'playing', 'guitar'], ['man', 'cutting', 'an', 'onion'], ['man', 'cycling'], ['man', 'slicing', 'open', 'fish'], ['man', 'slicing', 'tomato'], ['man', 'playing', 'guitar'], ['baby', 'panda', 'goes', 'down', 'slide'], ['man', 'singing', 'playing', 'guitar'], ['man', 'attacks', 'woman'], ['man', 'driving', 'car'], ['woman', 'cutting', 'tofu'], ['woman', 'styling', 'hair'], ['zebras', 'play', 'an', 'open', 'field'], ['man', 'cutting', 'potato'], ['man', 'slicing', 'an', 'onion'], ['man', 'dancing'], ['man', 'riding', 'motorcycle'], ['woman', 'slicing', 'garlics'], ['man', 'speaking'], ['little', 'boy', 'singing', 'playing', 'guita

In [ ]:
# wikiからの単語リストの作成
from collections import defaultdict

def wiki_tokenizer(filename):
    i = 0
    word2freq = defaultdict(int) # 出現回数を格納
    word2docnum = defaultdict(int) # 何文書に登場したかを格納
    with open(filename, "r") as f:
        for sentence in f:
            if i == 1000000:
                break
            s = tokenizer(sentence.lower())
            for token in s:
                word2freq[token] += 1
            for token in set(s):
                word2docnum[token] += 1
            i += 1
    return word2freq, word2docnum

word2freq, word2docnum = wiki_tokenizer("/content/drive/MyDrive/Vdata/English/en_wiki.sent.10m.txt")

In [ ]:
# 確率の計算
def cal_p(word2freq):
    word2prob = defaultdict(float)
    total = sum(word2freq.values())
    for word in word2freq:
        word2prob[word] = word2freq[word] / total
    return word2prob

word2prob = cal_p(word2freq)

In [ ]:
# ベクトルの重み付き平均の算出
import numpy as np

def avg_embedding(word2vec, word2prob):
    vector = np.zeros(word2vec.vector_size)
    for word in word2vec.vocab:
        if word in word2prob:
            vector += word2prob[word] * word2vec[word]
    return vector
    
avg_w2v_1 = avg_embedding(w2v_1, word2prob)
avg_w2v_2 = avg_embedding(w2v_2, word2prob)

In [ ]:
# w2v_1 中心化
import numpy as np

vector1_1 = list()
vector2_1 = list()

def vectorize(words, w2v):
    vectors = list()
    for word in words:
        if word in w2v:
            vectors.append(w2v[word] - avg_w2v_1)
    return np.array(vectors).mean(axis=0)

for i in range(textnum):
    vector1_1.append(vectorize(wordslist1[i], w2v_1))
    vector2_1.append(vectorize(wordslist2[i], w2v_1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# w2v_2 中心化
import numpy as np

vector1_2 = list()
vector2_2 = list()

def vectorize(words, w2v):
    vectors = list()
    for word in words:
        if word in w2v:
            vectors.append(w2v[word] - avg_w2v_2)
    return np.array(vectors).mean(axis=0)

for i in range(textnum):
    vector1_2.append(vectorize(wordslist1[i], w2v_2))
    vector2_2.append(vectorize(wordslist2[i], w2v_2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice.
  if sys.path[0] == '':


In [ ]:
# BoW
vector1_bow = list()
vector2_bow = list()

# 語彙（扱いたい全ての単語）
vocab = set()
for (words1, words2) in zip(wordslist1, wordslist2):
    for (word1, word2) in zip(words1, words2):
        vocab.add(word1)
        vocab.add(word2)

# 各単語にIDを割り当てる
word2id = dict()
for word in vocab:
    word2id[word] = len(word2id)

    def vectorize(words, vocab, word2id):
        vector = np.zeros(len(vocab))
        for word in words:
            if word in vocab:
                vector[word2id[word]] = 1
        return vector

for i in range(textnum):
    vector1_bow.append(vectorize(wordslist1[i], vocab, word2id))
    vector2_bow.append(vectorize(wordslist2[i], vocab, word2id))

In [ ]:
# 類似度推定: 2つのベクトル -> コサイン類似度
import math

def cos(v1, v2):
    if math.isnan(np.linalg.norm(vector1_1[i]) * np.linalg.norm(vector2_1[i])):
        c = 1
    else:
        c = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return c

f = open("Engw2v.txt", "w", encoding="UTF-8")

for i in range(textnum):
    ensamble = (0.4 * cos(vector1_1[i], vector2_1[i]) + 0.3 * cos(vector1_2[i], vector2_2[i]) + 0.3 * cos(vector1_bow[i], vector2_bow[i]))
    print("%1.3f\t%s\t%s" % (ensamble, wordslist1[i], wordslist2[i]))
    f.write(str(ensamble)+"\n")

f.close()

0.742	['girl', 'styling', 'hair']	['girl', 'brushing', 'hair']
0.818	['group', 'men', 'play', 'soccer', 'beach']	['group', 'boys', 'playing', 'soccer', 'beach']
0.855	['woman', 'measuring', 'another', 'woman', 'ankle']	['woman', 'measures', 'another', 'woman', 'ankle']
0.825	['man', 'cutting', 'cucumber']	['man', 'slicing', 'cucumber']
0.705	['man', 'playing', 'harp']	['man', 'playing', 'keyboard']
0.698	['woman', 'cutting', 'onions']	['woman', 'cutting', 'tofu']
0.842	['man', 'riding', 'an', 'electric', 'bicycle']	['man', 'riding', 'bicycle']
0.775	['man', 'playing', 'drums']	['man', 'playing', 'guitar']
0.818	['man', 'playing', 'guitar']	['lady', 'playing', 'guitar']
0.747	['man', 'playing', 'guitar']	['man', 'playing', 'trumpet']
0.747	['man', 'playing', 'guitar']	['man', 'playing', 'trumpet']
0.854	['man', 'cutting', 'an', 'onion']	['man', 'cuts', 'an', 'onion']
0.463	['man', 'cycling']	['man', 'talking']
0.726	['man', 'slicing', 'open', 'fish']	['man', 'cutting', 'fish']
0.703	['m